In [1]:
!pip install PyPDF2 -q
!pip install fitz -q
!pip install PyMuPDF -q

In [2]:
import PyPDF2
import glob
import fitz
import json
import re

In [3]:
def find_files(path_to_folder: str = '/content'):
    pdf_files = glob.glob(path_to_folder + '/*.pdf')
    #pdf_files = glob.glob(folder_path + '/**/*.pdf', recursive=True) для рекурсивного поиска по сабдиректориям
    return pdf_files

In [4]:
def find_labels(pdf_file_):
    page_num = 1
    labels = {}
    doc = fitz.open(pdf_file_)
    page = doc.load_page

    for page in doc:
        page_labels = []
        for annot in page.annots():
            text = page.get_textbox(annot.rect)
            page_labels.append(text)
        labels[page_num] = page_labels
        page_num += 1
    return labels



def pdf2text(pdf_files):
    data = {}
    for pdf_file_ in pdf_files:
        file_map = {}

        labels = find_labels(pdf_file_)

        pdf_file = open(pdf_file_, 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        for page in range(num_pages):
            pdf_page = pdf_reader.pages[page]
            pdf_text = pdf_page.extract_text()

            file_map[page + 1] = {'text': pdf_text, 'labels': labels[page + 1]}
        data[pdf_file_[9:]] = file_map

        pdf_file.close()

    return data

In [16]:
def find_theme(text):
    text = text.replace('\n', '')
    matches = re.findall(r'«(.+?)»', text)

    len_counter = 0
    chosed_text = ''
    for text in matches:
        if len(text) > len_counter:
            len_counter = len(text)
            chosed_text = text
    return chosed_text

In [6]:
pdf_files = find_files()

In [7]:
data = pdf2text(pdf_files)